In [ ]:
import pandas as pd
import numpy as np

from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

from dahuffman import HuffmanCodec

In [ ]:
df = pd.read_csv("filled/transact_18_22.csv")
df.set_index(['client', 'date'], inplace=True)
df.index = df.index.set_levels(pd.to_datetime(df.index.levels[1]), level=1)
df = df.sort_index()
df

In [ ]:
def compute_huffman_ratios(df: pd.DataFrame, columns: list):
    clients = df.index.get_level_values(0).unique()
    results = []

    for client in tqdm(clients, desc="Processing clients"):
        client_data = df.xs(client, level=0)
        row = {}

        for col in columns:
            seq = client_data[col].astype(str).tolist()

            codec = HuffmanCodec.from_data(seq)
            encoded = codec.encode(seq)
            original_size = len(seq) * 3
            compressed_size = len(encoded) * 8
            ratio = (compressed_size / original_size)

            row[f"huffman_ratio_{col}"] = ratio

        results.append(pd.Series(row, name=client))

    df_result = pd.DataFrame(results)
    df_result.index.name = "client"
    return df_result


In [ ]:
huffman_df = compute_huffman_ratios(df, ["code"])
huffman_df

In [ ]:
def plot_huffman_distributions(df_huffman):
    num_cols = len(df_huffman.columns)
    fig, axes = plt.subplots(num_cols, 1, figsize=(8, 4*num_cols))

    if num_cols == 1:
        axes = [axes]

    for ax, col in zip(axes, df_huffman.columns):
        sns.histplot(df_huffman[col], bins=30, kde=True, ax=ax, color='skyblue')
        ax.set_title(f'Distribution of huffman_ratio', fontsize=14)
        ax.set_xlabel('Huffman Ratio')
        ax.set_ylabel('Clients')
        ax.set_xlim(0, max(df_huffman[col].max()*1.1, 1.0))

    plt.tight_layout()
    plt.show()

In [ ]:
plot_huffman_distributions(huffman_df)